In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Initialize MediaPipe face detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

def preprocess_face(image):
    try:
        # Convert the image from float32 back to uint8
        image = np.uint8(image * 255.0)
        with mp_face_detection.FaceDetection(min_detection_confidence=0.2) as face_detection:
            # Convert the image from BGR to RGB as MediaPipe uses RGB images
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Process the image and detect faces
            results = face_detection.process(image_rgb)

            if results.detections:
                for detection in results.detections:
                    # Extract bounding box information
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = image.shape
                    bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                           int(bboxC.width * iw), int(bboxC.height * ih)

                    # Extract the face region
                    face = image[bbox[1]:bbox[1] + bbox[3], bbox[0]:bbox[0] + bbox[2]]

                    # Resize the face region to the desired size (224x224)
                    face_resized = cv2.resize(face, (224, 224))

                    return face_resized

        # If no faces are detected, return the original resized image
        return cv2.resize(image, (224, 224))
    except Exception as e:
        print(f"Error in preprocess_face: {e}")
        return cv2.resize(image, (224, 224))

def preprocess_function(image):
    image = preprocess_face(image)
    return image / 255.0  # Normalize the image

# Create ImageDataGenerators for training and validation datasets with augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_function,

)

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_function
)

# Load training dataset from person1 and person2 directories
train_dataset = train_datagen.flow_from_directory(
    'dataset/train/',
    target_size=(224, 224),
    batch_size=8,
    class_mode='binary'
)

# Load validation dataset from person1 and person2 directories
validation_dataset = validation_datagen.flow_from_directory(
    'dataset/validation/',
    target_size=(224, 224),
    batch_size=8,
    class_mode='binary'
)

# Load the VGG16 model with pre-trained weights from the local file and exclude the top layers
base_model = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))
base_model.load_weights('weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

# Add new top layers for fine-tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['acc'])

# Fine-tune the model
model_fit = model.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // 8,
    epochs=20,
    validation_data=validation_dataset,
    validation_steps=validation_dataset.samples // 8
)


In [ ]:
# Function to predict and display image with predicted class
def predict_and_display_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    plt.imshow(img)
    plt.show()
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    X = tf.keras.applications.vgg16.preprocess_input(X)
    
    # Predict the class
    prediction = model.predict(X)
    print(f"Raw model output: {prediction[0][0]}")
    
    # Determine the class based on prediction confidence
    if prediction[0][0] < 0.1:
        print("Predicted class: Marcus")
    elif prediction[0][0] > 0.9:
        print("Predicted class: Ronaldo")
    else:
        print("Predicted class: Unknown")

# Predicting on new images in the test folder
test_dir = 'dataset/test'

for i in os.listdir(test_dir):
    img_path = os.path.join(test_dir, i)
    if img_path.endswith(('.png', '.jpg', '.jpeg')):
        predict_and_display_image(img_path)